<div style="width:100%; text-align:center; margin: 40px 0;">
    <h1 style="color: #003366; font-size: 40px; font-weight:bold; text-shadow: 2px 2px 5px grey;">
        Énergie Électrique en France
    </h1>
    <hr style="width: 50%; height: 3px; background-color: #003366; border: none;">
    <p style="font-style:italic; font-size:18px; color: #555;">
        Une exploration approfondie de l'énergie électrique en France
    </p>
</div>


## Données:

Dans ce projet nous allons travailler sur des données produites par RTE France. Nous allons récolter ces données grâce à une API (OpenData Reseaux Energie https://opendata.reseaux-energies.fr/). Nous allons travailler sur des données régionales récoltées au niveau régional et sur des données nationales. Nous aurons aussi des API qui contiennent des données en temps réel, ce qui nous permettra d'avoir des données mise à jour régulierement.  

Voici les 4 liens API que nous utiliserons durant ce projet:
- Données Nationales en temps réel (du 1er Juin 2022 à aujourd'hui): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-national-tr`
- Données Nationales fixes (jusqu'au 31 Mai 2022): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-national-cons-def`
- Données Régionales en temps réel (du 1er Juin 2022 à aujourd'hui): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-tr`
- Données Régionales fixes (jusqu'au 31 Mai 2022): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-cons-def`

Grâce au module `request` de python nous allons pouvoir faire des requêtes de type SQL à ces API dans le but d'obtenir les données dont nous avons besoin au format JSON.

Nous utiliserons aussi un fichier GeoJson récupéré sur le site `https://france-geojson.gregoiredavid.fr/`. Ce fichier possède les données géographiques de la France, par région ou par département. Les données de RTE France étant uniquement nationales et régionales, nous utiliserons uniquement le GeoJson des régions de la France.

### Variables que nous allons analyser

- **Date** : *Date à laquelle les données ont ete enregistrees*
- **Heure** : *Date à laquelle les données ont ete enregistrees*
- **Région** (pour les données régionales) : *La région correspondant aux données*
- **Consommation** (MW) : *La consommation électrique réelle*
- **Prévision**  Conso J et J-1 (MW) :  *La consommation électrique prédite par RTE France au jours J et J-1*
- **Production par filières** (MW) : *La production d'energie electrique pqr filiere (nucleaire, hydrolique, eolien, solaire...)*
- **Imports exports  avec plusieurs pays** (MW) : *Echanges d'energie avec les differents pays frontaliers (Suisse, Allemagne, Belgique, Angleterre, Espagne)*
- **Emission de CO2** (g/kWh) : *La valeur d'emission de CO2*

## Type d'analyse prévue

Nous allons effectuer plusieurs visualisations sur nos données afin d'en tirer des observations. Pour cela nous allons principalement utiliser le module `plotly express` qui permet de faire des graphiques interactifs. Nous allons également améliorer l'interactivitée de notre graphique en rajoutant des modules IPython permetant à l'utilisateur de pouvoir trier et filtrer les données.  
Nous effecturons également des analyses statistiques afin de repondre à certains questionnements sur l'électricitée en France.

Pour cela nous concentrerons notre analyse sur 3 axes principaux:
- La consommation électrique en France
- La production d'énergie électrique aux niveaux national et régional
- Les échanges commerciaux avec les pays frontaliers

Nous allons d'abord devoir récupérer, nettoyer et formater les données lors des requêtes à l'API d'OpenData. Nous utiliserons ensuite Python et les bibliothèques Pandas et Plotly pour manipuler ces données et créer des visualisations.

## Résultats attendus

A la fin de cette analyse, nous devrions être capable de répondre à plusieurs questions tel que:
- Quels sont les modes de production les plus importants en France ? Par régions ?
- Comment a évolué la part du renouvelable dans le mix énergétique français durant la dernière décennie ?
- Quel est la balance commerciale de la France pour l’énergie ?
- Comment évolue la production et la consommation de l’électricité  en France  durant 1 journée ? 1 mois ? 1 année ?

Voici le type de graphique que l'on pourrait avoir à la fin de notre analyse:  

Voici un graphique représentant la consommation d'énergie électrique en France sur une seule journée.

![Consommation électrique](IMG/conso.png)

Ensuite nous pourrions avoir un graphique qui montre les échanges d'électricité avec les pays frontaliers.  
  
![Echanges commerciaux](IMG/echanges_comm.png)

Ensuite voici un graphique qui montre la proction d'energie électrique par la filière solaire sur une journée. Nous pouvons voir que le pic de production se situe vers 12h ce qui correspond au moment ou il y a le plus de soleil dans la journée.  
  
![Production d'energie solaire](IMG/production_solaire.png)  

Et pour finir voici un graphique representant la part de production d'énergie électrique pour chaque filiere:  
  
![Production d'energie par filiere](IMG/pie_chart_production.png)  
